First, import needed modules

In [ ]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [ ]:
path = os.getcwd()
print(path)

Import datasets

In [ ]:
btw02_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW02_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW13_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_es = pd.read_csv(f'{path}/data/elections/btw/es/btw17_wbz_erststimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str, 'Kennziffer Briefwahlzugehörigkeit': str})

btw02_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW02_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw05_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW05_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw09_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW09_Zweitstimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW13_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw17_zs = pd.read_csv(f'{path}/data/elections/btw/zs/btw17_wbz_zweitstimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str, 'Kennziffer Briefwahlzugehörigkeit': str})

View the data

In [ ]:
btw17_zs.head()

Add year column to dataframes

In [ ]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es, btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
years = [2002, 2005, 2009, 2013, 2017, 2002, 2005, 2009, 2013, 2017]
for df, year in zip(dfs, years):
    df.insert(loc=0, column='year', value=year)

In [ ]:
btw17_es.head()

Since some municipalities share a ballot vote election board, in this case, postal ballots cannot be attributed to a single municipality. 
In the 2002 to 2017 data, these cases were signified with '999' as Gemeindekennziffer. From 2017 on the Gemeindekennziffer is given by 9 + Kennziffer Briefwahlzugehörigkeit.

To drop the 999 Gemeinde in for 2002-2013, first append these dataframes

In [ ]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es]
btw0213_es = pd.concat(dfs, sort=False)
dfs = [btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
btw0213_zs = pd.concat(dfs, sort=False)
btw0213_zs.head()

Then drop all observations with Gemeinde == 999

In [ ]:
btw0213_es = btw0213_es[btw0213_es['Gemeinde'] != '999']
btw0213_zs = btw0213_zs[btw0213_zs['Gemeinde'] != '999']

For 2017, drop all observations where Gemeinde == 9 + Kennziffer Briefwahlzugehörigkeit

In [ ]:
btw17_es = btw17_es[btw17_es['Gemeinde'] != '9' + btw17_es['Kennziffer Briefwahlzugehörigkeit']]
btw17_zs = btw17_zs[btw17_zs['Gemeinde'] != '9' + btw17_zs['Kennziffer Briefwahlzugehörigkeit']]

Now contatenate the dfs for 2002-13 with 2017

In [ ]:
dfs = [btw0213_es, btw17_es]
btw_es = pd.concat(dfs, sort=False)
dfs = [btw0213_zs, btw17_zs]
btw_zs = pd.concat(dfs, sort=False)

Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [ ]:
btw_es.insert(loc=0, column='AGS', value=0)
btw_es['AGS'] = btw_es['Land'] + btw_es['Regierungsbezirk'] + btw_es['Kreis'] + btw_es['Gemeinde']
btw_zs.insert(loc=0, column='AGS', value=0)
btw_zs['AGS'] = btw_zs['Land'] + btw_zs['Regierungsbezirk'] + btw_zs['Kreis'] + btw_zs['Gemeinde']
btw_zs.head()

Collapse data along AGS column, sums votes over all constituencies in a municipality

In [ ]:
btw_es = btw_es.groupby(['AGS', 'year', 'Land'], as_index=False).sum()
btw_zs = btw_zs.groupby(['AGS', 'year', 'Land'], as_index=False).sum()
btw_zs.head()

Add indicator for Erst- und Zweitstimme

In [ ]:
dfs = [btw_es, btw_zs]
eszs = ['Erststimme', 'Zweitstimme']
for df, stimme in zip(dfs, eszs):
    df.insert(loc=2, column=stimme, value=1)

In [ ]:
btw_zs.head()

ES: Sum votes for other, Union and Die Linke parties

In [ ]:
# list(btw_es.columns)

In [ ]:
andere = ['REP',
'GRAUE',
'TIERSCHUTZ',
'NPD',
'ÖDP',
'PBC',
'FRAUEN',
'BP',
'FAMILIE',
'CM',
'BÜSO',
'HP',
'VIOLETTEN',
'AUFBRUCH',
'ZENTRUM',
'KPD',
'PRG',
'SCHILL',
'DEUTSCHLAND',
'DKP',
'DSU',
'FP D',
'ÜBRIGE',
'Offensive D',
'Die Tierschutzpartei',
'DIE FRAUEN',
'BüSo',
'Deutschland',
'AGFG',
'APPD',
'MLPD',
'DIE PARTEI',
'HUMANWIRTSCHAFTSPARTEI',
'STATT Partei',
'UNABHÄNGIGE',
'Übrige', 
'Volksabstimmung',
'ADM',
'DIE VIOLETTEN',
'ödp',
'PIRATEN',
'RRP',
'Freie Union',
'Tierschutzpartei',
'Bündnis 21/RRP',
'RENTNER',
'AfD',
'BIG',
'pro Deutschland',
'FREIE WÄHLER',
'PARTEI DER VERNUNFT',
'Die PARTEI',
'Bergpartei',
'BGD',
'NEIN',
'WGr/EB',
'PDV',
'SGP',
'DIE RECHTE',
'Tierschutzallianz',
'B*',
'Die Grauen',
'du',
'MG',
'MENSCHLICHE WELT',
'Gesundheitsforschung',
'V-Partei³',
'Bündnis C',
'DIE EINHEIT',
'MIETERPARTEI',
'Neue Liberale']
btw_es['Andere'] = btw_es[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_es['Union'] = btw_es[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_es['Linke'] = btw_es[linke].sum(axis=1) # sum votes for 'Linke' Parties
invalid = ['Ungültige', 'Ungültig']
btw_es['Ungültig'] = btw_es[invalid].sum(axis=1) # sum invalid votes

ZS: Sum votes for other, Union and DIE LINKE parties

In [ ]:
# list(btw_zs.columns)

In [ ]:
andere = ['REP',
'GRAUE',
'Tierschutz',
'NPD',
'ödp',
'PBC',
'FRAUEN',
'BP',
'FAMILIE',
'CM',
'BüSo',
'HP',
'Violetten',
'AUFBRUCH',
'ZENTRUM',
'KPD',
'PRG',
'Schill',
'Offensive D',
'Die Tierschutzpartei',
'DIE FRAUEN',
'Deutschland',
'AGFG',
'APPD',
'50plus',
'MLPD',
'Die Partei',
'PSG',
'Pro DM',
'Volksabstimmung',
'ADM',
'DKP',
'DVU',
'DIE VIOLETTEN',
'FWD',
'PIRATEN',
'RRP',
'RENTNER',
'Tierschutzpartei',
'ÖDP',
'Bündnis 21/RRP',
'AfD',
'BIG',
'pro Deutschland',
'DIE RECHTE',
'FREIE WÄHLER',
'Partei der Nichtwähler',
'PARTEI DER VERNUNFT',
'Die PARTEI',
'PDV',
'SGP',
'AD-DEMOKRATEN',
'Tierschutzallianz',
'B*',
'BGE',
'DiB',
'DM',
'Die Grauen',
'du.',
'MG',
'MENSCHLICHE WELT',
'Die Humanisten',
'Gesundheitsforschung',
'V-Partei³']
btw_zs['Andere'] = btw_zs[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_zs['Union'] = btw_zs[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_zs['Linke'] = btw_zs[linke].sum(axis=1) # sum votes for 'Linke' Parties
invalid = ['Ungültige', 'Ungültig']
btw_zs['Ungültig'] = btw_zs[invalid].sum(axis=1) # sum invalid votes

Concatenate Erst- and Zweitstimme datasets

In [ ]:
dfs = [btw_es, btw_zs]
btw = pd.concat(dfs, sort=False)
btw = btw.sort_values(['AGS', 'year'])
btw.head(10)

Add post-treatment indicator

In [ ]:
years = [2017, 2013, 2009, 2005]
for year in years:
    btw.insert(loc=3, column=f'post_{year}', value=0)
    btw[f'post_{year}'] = np.where(btw['year'] >= year, 1, 0)
btw.head(10)

Drop not needed columns and rename

In [ ]:
btw = btw[['AGS', 'year', 'post_2005', 'post_2009', 'post_2013', 'post_2017', 'Land', 'Erststimme', 'Zweitstimme', 'Wahlberechtigte (A)', 'Wähler (B)', 'Ungültig', 'Union', 'SPD', 'FDP', 'Linke', 'GRÜNE', 'Andere']]
btw.rename(columns = {'Wahlberechtigte (A)':'Wahlberechtigte', 'Wähler (B)': 'Wähler', 'GRÜNE': 'Grüne'}, inplace = True)
btw.head(10)

Check summation of votes

In [ ]:
parties = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'Ungültig']
btw['votes'] = btw[parties].sum(axis=1)
btw['votes'] = btw['votes'].astype(int) #convert to integer
btw['ok'] = btw['Wähler'].equals(btw['votes'])  # check if votes == Wähler
print(btw['ok'].describe())
btw.drop(['votes', 'ok', 'Ungültig'], axis=1, inplace=True)

Calculate vote shares in percent

In [ ]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    btw.loc[:, party] = (btw[party] / btw['Wähler'])*100
btw.loc[:, 'Wähler'] = (btw['Wähler'] / btw['Wahlberechtigte'])*100
btw.head(10)

Export dataset

In [ ]:
btw = btw.set_index(['AGS', 'year'])
btw.to_csv(f'{path}/data/btw.csv', encoding = 'utf-8-sig')
btw.head()